In [ ]:
# 1. Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# 2. Chargement du dataset
file_path = 'listings.csv'  # Remplacer par le chemin de votre fichier
df = pd.read_csv(file_path)

# Affichage des premières lignes pour vérifier
print(df.head())

# 3. Sélection des colonnes pertinentes
colonnes_pertinentes = [
    'price', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'beds',
    'bathrooms_text', 'neighbourhood_cleansed', 'host_is_superhost',
    'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating',
    'instant_bookable'
]
df = df[colonnes_pertinentes]

# 4. Suppression des doublons
df_cleaned = df.drop_duplicates()

# 5. Nettoyage des données
# Nettoyage de la colonne "price" pour enlever les symboles monétaires et convertir en numérique
df_cleaned['price'] = df_cleaned['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# 6. Gestion des valeurs manquantes
# Imputation avec RandomForest pour les variables numériques
def impute_with_random_forest(df_cleaned, target_column, features):
    # Séparation entre données avec et sans valeur manquante pour la cible
    df_train = df_cleaned.dropna(subset=[target_column])
    df_missing = df_cleaned[df_cleaned[target_column].isna()]

    # Sélection des variables explicatives (features)
    X_train = df_train[features]
    y_train = df_train[target_column]

    # Entraînement d'un modèle RandomForest
    rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_regressor.fit(X_train, y_train)

    # Prédiction des valeurs manquantes
    X_missing = df_missing[features]
    predicted_values = rf_regressor.predict(X_missing)

    # Imputation des valeurs manquantes
    df_cleaned.loc[df_cleaned[target_column].isna(), target_column] = predicted_values

    return df_cleaned

# Appliquer RandomForest pour l'imputation de valeurs manquantes dans "price"
df_cleaned = impute_with_random_forest(df_cleaned, 'price', ['accommodates', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating'])

# Appliquer RandomForest pour l'imputation des variables catégorielles "property_type" et "room_type"
df_cleaned = impute_with_random_forest(df_cleaned, 'property_type', ['accommodates', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating'])
df_cleaned = impute_with_random_forest(df_cleaned, 'room_type', ['accommodates', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating'])

# 7. Conversion des types de données (en particulier les variables catégorielles)
df_cleaned['host_is_superhost'] = df_cleaned['host_is_superhost'].map({'t': 'Oui', 'f': 'Non'}).astype('category')
df_cleaned['instant_bookable'] = df_cleaned['instant_bookable'].map({'t': 'Oui', 'f': 'Non'}).astype('category')

# 8. Transformation de la variable "bathrooms_text" en valeur numérique
df_cleaned['bathrooms_text'] = df_cleaned['bathrooms_text'].apply(lambda x: 1 if 'half' in str(x) else 0)

# 9. Normalisation des données
scaler = StandardScaler()
colonnes_numeriques = ['price', 'accommodates', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating']
df_cleaned[colonnes_numeriques] = scaler.fit_transform(df_cleaned[colonnes_numeriques])

# 10. Encodage des variables catégorielles
# Utilisation de l'encodage One-Hot pour les variables catégorielles
df_encoded = pd.get_dummies(df_cleaned, drop_first=True)

# 11. Séparation des données en jeu d'entraînement et de test
X = df_encoded.drop('price', axis=1)  # Variables explicatives
y = df_encoded['price']  # Variable cible

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 12. Construction du modèle de réseau de neurones avec des ajustements supplémentaires
model = Sequential()

# Ajout de la couche d'entrée et des premières couches cachées avec Dropout
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))  # Dropout pour éviter le surapprentissage

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# Ajout de la couche de sortie (régression donc pas d'activation)
model.add(Dense(1))

# Compilation du modèle avec une fonction d'optimisation améliorée
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# tf optimizer a chercher

# 13. Ajout de l'early stopping pour éviter le sur-apprentissage
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 14. Entraînement du modèle avec réduction du taux d'apprentissage en cas de stagnation
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Lancer l'entraînement
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[reduce_lr, early_stop], verbose=1)

# 15. Prédictions sur le jeu de test
y_pred_nn = model.predict(X_test)

# 16. Calcul du R² et de l'erreur quadratique moyenne (RMSE) pour le réseau de neurones
r2_nn = r2_score(y_test, y_pred_nn)
rmse_nn = np.sqrt(mean_squared_error(y_test, y_pred_nn))

print(f'R² (Réseau de Neurones): {r2_nn:.4f}')
print(f'RMSE (Réseau de Neurones): {rmse_nn:.4f}')

# 17. Visualisation de l'évolution de la perte pendant l'entraînement
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Perte d\'entraînement et de validation')
plt.xlabel('Époques')
plt.ylabel('Perte')
plt.legend()
plt.show()


     id                        listing_url       scrape_id last_scraped  \
0  3109  https://www.airbnb.com/rooms/3109  20241206031559   2024-12-07   
1  5396  https://www.airbnb.com/rooms/5396  20241206031559   2024-12-09   
2  7397  https://www.airbnb.com/rooms/7397  20241206031559   2024-12-08   
3  7964  https://www.airbnb.com/rooms/7964  20241206031559   2024-12-09   
4  9359  https://www.airbnb.com/rooms/9359  20241206031559   2024-12-09   

            source                                             name  \
0      city scrape                                     zen and calm   
1      city scrape     Your perfect Paris studio on Île Saint-Louis   
2      city scrape                 MARAIS - 2ROOMS APT - 2/4 PEOPLE   
3  previous scrape                     Sunny apartment with balcony   
4      city scrape  Cozy, Central Paris: WALK or VELIB EVERYWHERE !   

                                         description  \
0  Lovely Appartment with one bedroom with a Quee...   
1  NEW SOF

C:\Users\FranckPERSONNE\AppData\Local\Temp\ipykernel_31512\901047985.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['price'] = df_cleaned['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)


ValueError: could not convert string to float: 'Entire rental unit'